In [ ]:
import torch
import torch.nn as nn
from torchvision.models import vgg16, VGG16_Weights
import torchvision.transforms as transforms

from PIL import Image
import numpy as np
import cv2
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import time

from pytorchyolo.models import load_model
from pytorchyolo.utils.transforms import Resize, DEFAULT_TRANSFORMS
from pytorchyolo.utils.utils import non_max_suppression

In [ ]:
vgg_model = vgg16(weights=VGG16_Weights.IMAGENET1K_FEATURES)

In [ ]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

In [ ]:
vgg_model

In [ ]:
#remove classifier
#reference: https://discuss.pytorch.org/t/how-to-delete-layer-in-pretrained-model/17648
vgg_model.classifier = Identity()

In [ ]:
vgg_model.eval()

In [ ]:
#  224x224
vgg_16_transform = transforms.Compose([
    transforms.Resize(224, interpolation=transforms.InterpolationMode.NEAREST),
    transforms.ToTensor()
])

In [ ]:
sample_img = Image.open('152018.jpg')
sample_img = vgg_16_transform(sample_img)
plt.imshow(transforms.ToPILImage()(sample_img), interpolation="bicubic")

sample_output = vgg_model(sample_img)
sample_output

In [ ]:
folders = ['2--Demonstration']

_, model = load_model('./weights/yolo_face_sthanhng.cfg', "./weights/yolo_face_sthanhng.weights")

ref_df = pd.read_csv(os.path.join(os.getcwd(), 'reference_dataset.csv'), index_col=0)
ref_df = ref_df.reset_index(drop=True)

OUTPUT_FOLDER = os.path.join(os.getcwd(), "WIDER")
if not os.path.exists(OUTPUT_FOLDER):
        os.mkdir(OUTPUT_FOLDER)
        
for FOLDER_NAME in folders:
    
    features_df = pd.DataFrame(columns=range(25088))
    
    INPUT_PATH = os.path.join(os.getcwd(), 'images', FOLDER_NAME)
    
    FOLDER_PATH = os.path.join(OUTPUT_FOLDER, FOLDER_NAME)
    if not os.path.exists(FOLDER_PATH):
        os.mkdir(FOLDER_PATH)
    
    CSV_PATH = os.path.join(FOLDER_PATH, FOLDER_NAME + '_CSV')
    if not os.path.exists(CSV_PATH):
        os.mkdir(CSV_PATH)
    
    for path in glob.glob(os.path.join(INPUT_PATH, '*.jpg')):
        filename = os.path.basename(path)
        
        filename_df = ref_df[ref_df['source_file'] == filename]
        
        if len(filename_df.index) > 0:
            model.eval()
            
            data = cv2.imread(path)  # read the image
            data = cv2.cvtColor(data, cv2.COLOR_BGR2RGB) #change to rgb
            data = transforms.Compose([DEFAULT_TRANSFORMS,Resize(416)])((data, np.zeros((1, 5))))[0].unsqueeze(0) # transform the image
            
            with torch.no_grad():
                # Forward pass the data through the model and call non max suppression
                nms, nms_output = non_max_suppression(model(data), 0.5, 0.5) #conf_thres and iou_thres = 0.5

            face_list = []
            if type(nms_output[0]) is not int:
                face_list = nms_output[0]
            
            for face_index, face_row in enumerate(face_list): #nms_output[0] because the model is designed to take in several images at a time from the dataloader but we are only loading the image one at a time
                if face_index in set(filename_df['face_index']):
                    
                    x, y, w, h = face_row[0], face_row[1], face_row[2], face_row[3]
                    
                    x1 = max(int(np.floor((x - w / 2).detach().cpu().numpy())), 0)
                    y1 = max(int(np.floor((y - h / 2).detach().cpu().numpy())), 0)
                    x2 = min(int(np.ceil((x + w / 2).detach().cpu().numpy())), 415)
                    y2 = min(int(np.ceil((y + h / 2).detach().cpu().numpy())), 415)
                    
                    cropped_image = data.detach().cpu()[:, :, y1:y2, x1:x2]
                    cropped_image = np.transpose(cropped_image.detach().cpu().squeeze().numpy(), (1, 2, 0))
                    cropped_image = (cropped_image * 255).astype(np.uint8)
                    cropped_image = Image.fromarray(cropped_image)
                    
                    plt.imshow(cropped_image, interpolation='nearest')
                    plt.show()
                    
                    cropped_image = vgg_16_transform(cropped_image)
#                     with torch.no_grad():
#                         vgg_output = vgg_model(cropped_image)

                    vgg_features = torch.flatten(vgg_output).tolist()
#                     print(type(vgg_features))
                    features_df.loc[len(features_df)] = vgg_features
    
    features_df.to_csv(os.path.join(CSV_PATH, FOLDER_NAME + '_VGG16_features' + str(int(time.time())) + '.csv'), index=False)  #save to csv

In [ ]:
# features_df.to_csv(os.path.join(CSV_PATH, FOLDER_NAME + '_VGG16_features' + str(int(time.time())) + '.csv'), index=False)  #save to csv